In [3]:
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output, State
import plotly
import plotly.graph_objs as go
import pandas as pd
from pathlib import Path
import os

# Import the sample data

In [86]:
DATA = Path(r"C:\Users\lhi30\Haein\2023\YBIGTA\2023-2\DA\Wiki_People\Share\data\ready_for_visualization")
csv_list = os.listdir(DATA)
df_dict = {}
for f in csv_list:
    df = pd.read_csv(DATA / f)
    name = f[:-4]
    df_dict[name] = df


In [87]:
df = df_dict['14th Dalai Lama']

In [88]:
df

,year,age,event,category1,category2
0,1937,2,14th Dalai Lama was selected as the tulku of t...,summary,NaN
1,1935,0,14th Dalai Lama was born on 6 July 1935 to a f...,Early life and background,NaN
2,1940,5,14th Dalai Lama's formal rule as head of the g...,Life as the Dalai Lama,Cooperation and conflicts with the People's Re...
3,1954,19,"On 27 September 1954, 14th Dalai Lama was sele...",Life as the Dalai Lama,Cooperation and conflicts with the People's Re...
4,1999,64,"In 14th Dalai Lama's essay ""The Ethic of Compa...",Life as the Dalai Lama,"Teaching activities, public talks"
5,1984,49,"As examples of other teachings, in London in 1...",Life as the Dalai Lama,"Teaching activities, public talks"
6,2007,72,14th Dalai Lama has met the Archbishop of Cant...,Life as the Dalai Lama,Interfaith dialogue
7,2010,75,"In 2010, 14th Dalai Lama, joined by a panel of...",Life as the Dalai Lama,Interfaith dialogue
8,2019,84,When asked in 2019 about attacks on the minori...,Social stances,"Democracy, nonviolence, religious harmony, and..."
9,1994,59,"In a 1994 interview with OUT Magazine, 14th Da...",Social stances,Sexuality


In [4]:
def generate_timeline(person_name: str, person_df: pd.DataFrame) -> plotly.graph_objects.Figure:
    """
    Generate a life timeline plot for a person based on the provided DataFrame.

    Parameters:
    - person_name (str): The name of the person.
    - person_df (pd.DataFrame): DataFrame containing ["year", "age", "event", "category1", "category2"].

    Returns:
    - plotly.graph_objects.Figure: The generated timeline plot.
    """

    def generate_hover_text(row:pd.DataFrame):
        ## Helper function to generate display information
        hover_text = row["event"]
        if pd.notna(row["category1"]):
            hover_text += f"<br><b>Heading:</b> {row['category1']}"
        if pd.notna(row["category2"]):
            hover_text += f"<br><b>Sub-Heading:</b> {row['category2']}"
        return hover_text
    # Given the information we have, let's generate what to display
    person_df["hover_text"] = person_df.apply(generate_hover_text, axis = 1)
    # Let's create the plot's title 
    figure_title = f"{person_name}'s Life, according to Wikipedia"
    # The actual graph figure!
    fig = go.Figure()
    
    # Add the data points
    fig.add_trace(
        go.Scatter(
            x=person_df["year"],
            y=[1] * len(person_df),
            mode="markers+text",
            marker=dict(size=20),
            text=person_df["year"].astype(str) + " - Age: " + person_df["age"].astype(str),  # Set text for the data points
            hoverinfo="text",  # Display the text
            hovertext=person_df["hover_text"], # When this is set, only this is shown on hoverinfo
            showlegend=False,  # Hide this trace in the legend
            textposition="bottom center",
            textfont=dict(color="black", size=10,),
        )
    )
    fig.update_layout(
        title=figure_title,
        xaxis=dict(showgrid=True, showline=True, showticklabels=True, ticks="outside"),
        yaxis=dict(showgrid=False, showline=False, showticklabels=False),
        hovermode="closest",  # Only display hover info for the closest data point
    )
    # fig.show()
    return fig

In [84]:
generate_timeline("Abbe Pierre", df_list[1])

# Make the plotly graph first

In [ ]:
name_figures = {}
for name, df in df_dict.items():
    figure = generate_timeline(name, df)
    name_figures[name] = figure

In [ ]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
import plotly.graph_objs as go

app = dash.Dash(__name__)

# Sample initial figure
initial_figure = go.Figure(data=[go.Scatter(x=[1, 2, 3], y=[4, 1, 2], mode='lines+markers')])

name_figures = {}
for name, df in df_dict.items():
    figure = generate_timeline(name, df)
    name_figures[name] = figure

app.layout = html.Div([
    # Row for the selected name's figure
    html.Div([
        # Selection bar at the top
        dcc.Dropdown(
            id='name-dropdown',
            options=[
                {'label': name, 'value': name} for name in name_figures.keys()
            ],
            value=list(name_figures.keys())[0],  # Set default value to the first name
            style={'width': '50%', 'margin-bottom': '20px'}
        ),

        # Placeholder for displaying the selected name's figure
        html.Div(id='selected-name-figure-placeholder', style={'width': '100%'})
    ], style={'width': '100%', 'margin-bottom': '20px'}),

    # Row for the initial figure
    html.Div([
        dcc.Graph(
            id='initial-figure',
            figure=initial_figure,
        ),
    ], style={'width': '100%'}),
])


# Callback to display the selected name's figure horizontally
@app.callback(
    Output('selected-name-figure-placeholder', 'children'),
    [Input('name-dropdown', 'value')]
)
def display_selected_name_figure(selected_name):
    if selected_name:
        selected_figure = name_figures.get(selected_name, initial_figure)
        return dcc.Graph(
            id='selected-name-figure',
            figure=selected_figure,
            style={'width': '100%'}
        )
    return None


if __name__ == '__main__':
    app.run_server(debug=True, use_reloader=False, mode='inline')


In [5]:
HOME_PATH = Path(r"C:\Users\lhi30\Haein\2023\YBIGTA\2023-2\DA\Wiki_People\Share")
def get_visualization_data(relative_path = "data/ready_for_visualization/") -> dict:
    """
    From the input path, return a dictionary od pandas DataFrames, with the csv files' names as the keys

    Parameters:
    - relative_path (str): The relative path of the folder with all the csv files ready for visualization

    Returns:
    - df_dict[name] = df: A dictionary with name(str) as the key, and the DataFrame as the value
    """
    get_path = HOME_PATH / relative_path
    csv_list = os.listdir(get_path)
    df_dict = {}
    for csv_file in csv_list:
        name = csv_file[:-4]
        df = pd.read_csv(HOME_PATH / relative_path / csv_file)
        df_dict[name] = df
    return df_dict

In [3]:
import dash
from dash import html

app = dash.Dash(__name__)

# Define the URL of your Tableau Public visualization with the embed parameter
tableau_url = "https://public.tableau.com/views/Metro_16986659627900/SeoulMetro?:language=en-US&:display_count=n&:origin=viz_share_link&:embed=y"

app.layout = html.Div([
    # Embed Tableau visualization using iframe
    html.Iframe(src=tableau_url, width="100%", height="600", style={"border": "none"}),
])

if __name__ == '__main__':
    app.run_server(debug=True, use_reloader=False, mode="inline")


: 